# Main built-in LCEL Runnables

## Contents
* RunnablePassthrough
* RunnableLambda
* RunnableParallel
    * itemgetter

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

## RunnablePassthrough

In [6]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough()

In [7]:
chain.invoke("Abram")

'Abram'

## RunnableLambda

In [8]:
def russian_lastname(name: str) -> str:
    return f"{name}ovich"

In [9]:
from langchain_core.runnables import RunnableLambda

chain = RunnablePassthrough() | RunnableLambda(russian_lastname)

In [10]:
chain.invoke("Abram")

'Abramovich'

## RunnableParallel

In [11]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "operation_b": RunnableLambda(russian_lastname)
    }
)

In [12]:
chain.invoke("Abram")

{'operation_a': 'Abram', 'operation_b': 'Abramovich'}

In [13]:
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "soccer_player": lambda x: x["name"]+"ovich"
    }
)

In [14]:
chain.invoke({
    "name1": "Jordam",
    "name": "Abram"
})

{'operation_a': {'name1': 'Jordam', 'name': 'Abram'},
 'soccer_player': 'Abramovich'}

### We can add more Runnables to chain

In [17]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [18]:
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

In [19]:
def russian_lastname_from_dictionary(person):
    return person["name"] + "ovich"

In [20]:
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "soccer_player": RunnableLambda(russian_lastname_from_dictionary),
        "operation_c": RunnablePassthrough(),
    }
) | prompt | model | output_parser

In [21]:
chain.invoke({
    "name1": "Jordam",
    "name": "Abram"
})

'One curious fact about Roman Abramovich is that he has a personal net worth estimated to be around $14 billion, making him one of the richest individuals in Russia. He made his fortune primarily through investments in oil and gas companies, as well as through his ownership of the Chelsea Football Club in the English Premier League. Despite his immense wealth, Abramovich is known for his philanthropic efforts, having donated millions of dollars to various charities and causes over the years.'

## More advanced use of RunnableParallel

In [ ]:
#!pip install faiss-cpu

In [24]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["AI Accelera has trained more than 7.000 Alumni from all continents and top companies"], embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(model="gpt-3.5-turbo")

retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("who are the Alumni of AI Accelera?")

'The Alumni of AI Accelera are individuals from all continents and top companies.'

## Using itemgetter with RunnableParallel

In [25]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

model = ChatOpenAI(model="gpt-3.5-turbo")

vectorstore = FAISS.from_texts(
    ["AI Accelera has trained more than 3,000 Enterprise Alumni."], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "How many Enterprise Alumni has trained AI Accelera?", "language": "Pirate English"})

'AI Accelera has trained more than 3,000 Enterprise Alumni. Arrr!'

## Important: Syntax of RunnableParallel can have several variations
* When composing RunnableParallel with another Runnable we do not need to wrap it up in RunnableParallel class. Inside chain, next three syntaxs are equivalent:
    * `RunnableParallel({"context": retriever, "question": RunnablePassthrough()})`
    * `RunnableParallel(context=retriever, question=RunnablePassthrough())`
    * `{"context": retriever, "question": RunnablePassthrough()}`